In [21]:
%load_ext autoreload

%autoreload 2

from __future__ import print_function, division

import cPickle as pickle
from glob import glob
import subprocess
import sys
import string
import os
import random
import numpy as np
from os.path import join

import threebody

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [168]:
basic_params = dict(files="tuned2_aligned_pulses_5deg",
                     tzrmjd_middle='auto',
                     parfile="0337_de430.par",
                     fit_pos=True,
                     fit_pm=True,
                     fit_px=True,
                     efac=1,
                     t2_astrometry=True,
                     kopeikin=False,
                     ppn_mode='heavysimple',
                     linear_jumps=True,
                     linear_dm=False,
                     fdn_range=(1,5),
                     priors=('dbeta','dgamma'),
                     toa_mode="pipeline",
                     dmx_span=365.2425/2,
                     variable_dm=True,
                     variable_ipm=True)


In [128]:
qsub_script = string.Template("""#!/bin/sh
#PBS -lnodes=1:ppn=1:compute:old
#PBS -N threebody_downhill_$name
#PBS -V
#PBS -o $DIR/pbs.out
#PBS -e $DIR/pbs.err

cd $here
export PATH=/users/aarchiba/.virtualenvs/triplesystem/bin:/users/aarchiba/.local/bin:/usr/local/src/presto/bin:/users/aarchiba/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/bin:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/users/aarchiba/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:.:/users/aarchiba/bin:/opt/local/share/cv/bin:/usr/local/bin:/opt/local/bin:/bin:/usr/bin:/bin:/usr/etc:/usr/sbin:/sbin:/usr/local/sbin:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/bin:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:/usr/lib64/qt-3.3/bin
export PYTHONPATH=

python downhill.py $DIR/fitter_params.pickle $DIR/best_values.pickle \
--method $method --best_parameters $sv > $DIR/stdout.txt
""")
def submit(params, name, method="BOBYQA", start=None):
    # FIXME: handle already-running jobs
    here = os.getcwd()
    DIR = join("downhill-runs",name)
    if not os.path.exists(DIR):
        os.makedirs(DIR)
    with open(join(DIR, "fitter_params.pickle"),"wb") as f:
        pickle.dump(params, f)
    F = threebody.Fitter(**params)
    if start is not None:
        F.best_parameters.update(start)
        sv = join(DIR, "start_values.pickle")
        with open(sv,"wb") as f:
            pickle.dump(F.best_parameters, f)
    else:
        sv = "default"
    for p in F.parameters:
        if p not in F.best_parameters:
            raise ValueError("best parameters don't include %s" % p)
    print("Initial goodness of fit:", F.goodness_of_fit(),
          "/", len(F.mjds))
    s = qsub_script.substitute(**locals())
    P = subprocess.Popen(["qsub","-"],stdin=subprocess.PIPE,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)
    out, err = P.communicate(input=s)
    if P.returncode:
        raise ValueError("qsub returned %s, output:\n%s\nerror:\n%s"
                         % (P.returncode, out, err))
    return out, err

In [161]:
start=None

In [196]:
jobid = "4070"
job_dir = glob(os.path.join("emcee-chains",jobid+".*"))[0]

with open(os.path.join(job_dir,"fitter_params.pickle")) as f:
    fitter_params = pickle.load(f)

F = threebody.Fitter(**fitter_params)
best_so_far = np.load(os.path.join(job_dir,"best_so_far.npy"))
start = F.best_parameters.copy()
for i,p in enumerate(F.parameters):
    start[p] = best_so_far[i]
G = threebody.Fitter(**basic_params)
print("MCMC setup before:", F.goodness_of_fit(), 
      "after:", F.goodness_of_fit(start))
print("basic_params before:", G.goodness_of_fit(),
      "after:", G.goodness_of_fit(start))

MCMC setup before: 39907.0546317 after: 39907.364184
basic_params before: 37937.4341033 after: 38087.1209711


In [197]:
submit(fitter_params, "MCMC", start=start)

Initial goodness of fit: 39907.364184 / 27138


('4182.nimrod.cv.nrao.edu\n', '')

In [169]:
p = basic_params.copy()
submit(p,"heavysimple",start=start)
#submit(p,"heavysimple",method="basinhopping",start=start)


Initial goodness of fit: 38051.0382286 / 27221


('4099.nimrod.cv.nrao.edu\n', '')

In [190]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_1400_nosun2"
submit(p,"lband",start=start)


Initial goodness of fit: 36364.2056757 / 27138


('4161.nimrod.cv.nrao.edu\n', '')

In [175]:
p = basic_params.copy()
p["ppn_mode"] = "GR"
submit(p,"GR")
#submit(p,"GR",start=start)
#submit(p,"GR",method="Powell",start=start)
#submit(p,"GR",method="basinhopping",start=start)


Initial goodness of fit: 37978.7130627 / 27221


('4105.nimrod.cv.nrao.edu\n', '')

In [171]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_5deg_WSRT"
submit(p,"WSRT",start=start)
#submit(p,"WSRT",method="Powell",start=start)
#submit(p,"WSRT",method="basinhopping",start=start)


Initial goodness of fit: 4147.26028511 / 3297


('4101.nimrod.cv.nrao.edu\n', '')

In [172]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_5deg_AO"
submit(p,"AO",start=start)
#submit(p,"AO",method="Powell",start=start)
#submit(p,"AO",method="basinhopping",start=start)


Initial goodness of fit: 10372.4280487 / 5437


('4102.nimrod.cv.nrao.edu\n', '')

In [173]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_5deg_GBT"
submit(p,"GBT",start=start)
#submit(p,"GBT",method="Powell")
#submit(p,"GBT",method="basinhopping")


Initial goodness of fit: 22842.5259149 / 18487


('4103.nimrod.cv.nrao.edu\n', '')

In [145]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_1400_nosun2"
p["linear_dm"] = True
p["variable_dm"] = False
p["variable_ipm"] = False
submit(p,"dmtesting",start=start)
#submit(p,"heavysimple",method="basinhopping",start=start)


Initial goodness of fit: 39950.1750333 / 27138


('4071.nimrod.cv.nrao.edu\n', '')

## Incorporate results into database

In [198]:
bpd = threebody.load_best_parameter_database()

for DIR in glob(join("downhill-runs","*")):
    name = DIR.split("/")[-1]
    d = pickle.load(open(join(DIR,"fitter_params.pickle")))
    F = threebody.Fitter(**d)
    try:
        p = pickle.load(open(join(DIR,"best_values.pickle")))
    except IOError:
        continue
    newp = F.best_parameters.copy()

    newp.update(p)
    if newp==F.best_parameters:
        print(name, "unchanged")
        continue
    try:
        old = F.goodness_of_fit(F.best_parameters)
    except KeyError:
        old = np.inf
    new = F.goodness_of_fit(newp)
    print(name, "existing best parameters:", old)
    print(name, "best parameters from fitter:", new)

    if new<old:
        bpd = threebody.load_best_parameter_database()
        print(name, "updating")
        bpd[F.bpd_k] = newp
        threebody.save_best_parameter_database(bpd)

WSRT unchanged
AO unchanged
GBT unchanged
heavysimple unchanged
GR unchanged
dmtesting existing best parameters: 39907.0546317
dmtesting best parameters from fitter: 39947.7999381
MCMC existing best parameters: 39907.0546317
MCMC best parameters from fitter: 39907.0546317
lband unchanged
